This is a script to compare the scdesigner AIC/BIC result to the scDesign3 AIC/BIC result.

In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

# scDesign3 impelentation

I will first calculate the AIC/BIC in scDesign3 under similar model setup.

In [12]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError("dlopen(/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <3D361A20-57F1-3C9B-9D2B-6A0F70CF5F35> /Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/pyl/Desktop/UW Madis

In [13]:
%%R
library(scDesign3)
library(ggplot2)
library(zellkonverter)
theme_set(theme_bw())

example_sce <- readH5AD("data/example_sce.h5ad")


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    ℹ Using stored X_name value 'X'


Registered S3 method overwritten by 'scDesign3':
  method         from  
  predict.gamlss gamlss
Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate


In [14]:
%%R
example_simu <- scdesign3(
  sce = example_sce,
  assay_use = "counts",
  celltype = "cell_type",
  pseudotime = "pseudotime",
  spatial = NULL,
  other_covariates = NULL,
  mu_formula = "s(pseudotime, k = 6, bs = 'cr')",
  sigma_formula = "1",
  family_use = "nb",
  n_cores = 2,
  usebam = FALSE,
  corr_formula = "1",
  copula = "gaussian",
  DT = TRUE,
  pseudo_obs = FALSE,
  return_model = FALSE,
  nonzerovar = FALSE
) 

Input Data Construction Start
Input Data Construction End
Start Marginal Fitting
Marginal Fitting End
Start Copula Fitting
Convert Residuals to Multivariate Gaussian
Converting End
Copula group 1 starts
Copula Fitting End
Start Parameter Extraction
Parameter
Extraction End
Start Generate New Data
Use Copula to sample a multivariate quantile matrix
Sample Copula group 1 starts
New Data Generating End


And below are the marginal and gaussian copula AIC/BIC calculated directly from scDesign3.

In [15]:
%%R
print(paste0("marginal AIC: ", example_simu$model_aic[1]))
print(paste0("marginal BIC: ", example_simu$model_bic[1]))
print(paste0("copula AIC: ", example_simu$model_aic[2]))
print(paste0("copula BIC: ", example_simu$model_bic[2]))

[1] "marginal AIC: 778898.707033089"
[1] "marginal BIC: 782773.03646551"
[1] "copula AIC: -8863.02610241459"
[1] "copula BIC: 19072.2142876176"


# scdesigner AIC/BIC example

Now we are going to extract AIC/BIC from our scdesigner models.

In [2]:
from scdesigner.simulators import NegBinCopulaSimulator

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `annda

In [8]:
formula = "~ bs(pseudotime, degree=5)"
nbcopula = NegBinCopulaSimulator()
nbcopula.fit(example_sce, formula)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


Marginal and copula AIC and BIC are included as instances within each simulator class. The AIC/BIC only using previous mean formula gives the same result as before.

In [9]:
print("Marginal AIC:", nbcopula.marginal_aic)
print("Marginal AIC:", nbcopula.marginal_bic)
print("Copula AIC:", nbcopula.copula_aic)
print("Copula BIC:", nbcopula.copula_bic)

Marginal AIC: 793849.9375
Marginal AIC: 797800.375
Copula AIC: -9314.957297915244
Copula BIC: 18620.2830921169


I will then compare the marginal AIC/BIC results on other distributions with generated datasets.

# Poisson example

In [1]:
from scdesigner.simulators import PoissonCopulaSimulator
from scipy.stats import poisson
import numpy as np
import pandas as pd
import anndata

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `annda

In [2]:
n_sample, n_gene, n_feature = 2000, 20, 2
X1 = np.random.normal(size=(n_sample, n_feature)) # covariates
ground_truth = np.random.normal(size=(n_feature, n_gene)) # feature x gene
beta = np.exp(X1 @ ground_truth) # cell x gene

# generate samples
Y = poisson(beta).rvs()
obs = pd.DataFrame(X1, columns=[f"dim{j}" for j in range(n_feature)]) # cell x feature
adata = anndata.AnnData(X=Y, obs=obs)
adata

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 2000 × 20
    obs: 'dim0', 'dim1'

In [3]:
formula = "~ dim0 + dim1 - 1"
pcopula = PoissonCopulaSimulator()
pcopula.fit(adata, formula)

print("marginal AIC:", pcopula.marginal_aic)
print("marginal BIC:", pcopula.marginal_bic)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]
Epoch 1/40: 0it [00:00, ?it/s]/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


marginal AIC: 109389.2421875
marginal BIC: 109613.28125


/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [4]:
beta_hat = np.exp(X1 @ pcopula.params['coef_beta'].values)
log_likelihood = np.sum(poisson(beta_hat).logpmf(Y))
k = n_feature * n_gene # nparams

print("marginal AIC:", 2 * k - 2 * log_likelihood)
print("marginal BIC:", k * np.log(n_sample) - 2 * log_likelihood)

marginal AIC: 109389.37310496412
marginal BIC: 109613.4092033458


# ZI-NB example

In [6]:
from scdesigner.simulators import ZeroInflatedNegBinCopulaSimulator
from scipy.stats import nbinom, bernoulli
from scipy.special import expit

In [7]:
n, g, d, p, z = 2000, 20, 2, 2, 2
X1 = np.random.normal(size=(n, d)) # 
X2 = np.random.normal(size=(n, p)) #  
X3 = np.random.normal(size=(n, z)) #  
B = np.random.normal(size=(d, g)) # feature x gene
D = np.random.normal(size=(p, g)) # feature x gene
Z = np.random.normal(size=(z, g)) # feature x gene
mu = np.exp(X1 @ B) # cell x gene
r = np.exp(X2 @ D) # cell x gene
pi = expit(X3 @ Z) # cell x gene

# generate samples
Y = nbinom(n=r, p=r / (r + mu)).rvs() * bernoulli(1 - pi).rvs()

X_1 = pd.DataFrame(X1, columns=[f"mean_dim{j}" for j in range(d)]) # cell x feature
X_2 = pd.DataFrame(X2, columns=[f"dispersion_dim{j}" for j in range(p)]) # cell x feature
X_3 = pd.DataFrame(X3, columns=[f"zero_inflation_dim{j}" for j in range(z)]) # cell x feature
obs = pd.concat([X_1, X_2, X_3], axis=1)

adata = anndata.AnnData(X=Y, obs=obs)

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [9]:
formula = {"mean": "~ mean_dim0 + mean_dim1 - 1", 
           "dispersion": "~ dispersion_dim0 + dispersion_dim1 - 1",
           "zero_inflation": "~ zero_inflation_dim0 + zero_inflation_dim1 - 1"}
zinb = ZeroInflatedNegBinCopulaSimulator()
zinb.fit(adata, formula)

print("marginal AIC:", zinb.marginal_aic)
print("marginal BIC:", zinb.marginal_bic)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


marginal AIC: 72666.8515625
marginal BIC: 73338.9609375


In [10]:
mu_hat = np.exp(X1 @ zinb.params['coef_mean'].values) # cell x gene
r_hat = np.exp(X2 @ zinb.params['coef_dispersion'].values) # cell x gene
pi_hat = expit(X3 @ zinb.params['coef_zero_inflation'].values) # cell x gene

log_likelihood = np.sum(
    np.log(pi_hat * (Y == 0) + (1 - pi_hat) * nbinom(n=r_hat, p=r_hat / (r_hat + mu_hat)).pmf(Y) + 1e-10))
k = g * (d+p+z) #

print("marginal AIC:", 2 * k - 2 * log_likelihood)
print("marginal BIC:", k * np.log(n) - 2 * log_likelihood)

marginal AIC: 72666.84474335308
marginal BIC: 73338.95303849813


# ZI-P example

In [11]:
from scdesigner.simulators import ZeroInflatedPoissonRegressionSimulator
from scipy.stats import poisson, bernoulli
from scipy.special import expit

In [12]:
n_sample, n_gene, n_feature1, n_feature2 = 2000, 20, 2, 3
X1 = np.random.normal(size=(n_sample, n_feature1)) # beta covariates
X2 = np.random.normal(size=(n_sample, n_feature2)) # zero-inflation covariates
gt_beta = np.random.normal(size=(n_feature1, n_gene))
gt_pi = np.random.normal(size=(n_feature2, n_gene))
beta = np.exp(X1 @ gt_beta)
pi = 1 / (1 + np.exp(-(X2 @ gt_pi)))

# generate samples
Y = poisson(beta).rvs() * bernoulli(1 - pi).rvs()
obs1 = pd.DataFrame(X1, columns=[f"beta_dim{j}" for j in range(n_feature1)])
obs2 = pd.DataFrame(X2, columns=[f"pi_dim{j}" for j in range(n_feature2)])
obs = pd.concat([obs1, obs2], axis=1)
adata = anndata.AnnData(X=Y, obs=obs)
adata

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 2000 × 20
    obs: 'beta_dim0', 'beta_dim1', 'pi_dim0', 'pi_dim1', 'pi_dim2'

In [13]:
formula = {"beta": "~ beta_dim0 + beta_dim1 - 1",
           "pi": "~ pi_dim0 + pi_dim1 + pi_dim2 - 1"}
zip = ZeroInflatedPoissonRegressionSimulator()
zip.fit(adata, formula)

print("marginal AIC:", zip.marginal_aic)
print("marginal BIC:", zip.marginal_bic)

marginal AIC: 74814.9921875
marginal BIC: 75375.0859375


In [14]:
beta_hat = np.exp(X1 @ zip.params['coef_beta'].values) # cell x gene
pi_hat = expit(X2 @ zip.params['coef_pi'].values) # cell x gene

log_likelihood = np.sum(
    np.log(pi_hat * (Y == 0) + (1 - pi_hat) * poisson(beta_hat).pmf(Y) + 1e-10))
k = n_gene * (n_feature1+n_feature2) # nparams

print("marginal AIC:", 2 * k - 2 * log_likelihood)
print("marginal BIC:", k * np.log(n_sample) - 2 * log_likelihood)

marginal AIC: 74814.96999869148
marginal BIC: 75375.06024464569


# Bernoulli example

In [15]:
from scdesigner.simulators import BernoulliCopulaSimulator

In [16]:
n_sample, n_gene, n_feature1 = 2000, 20, 2
X1 = np.random.normal(size=(n_sample, n_feature1)) # covariates
ground_truth = np.random.normal(size=(n_feature1, n_gene)) # feature x gene
beta = expit(X1 @ ground_truth) # cell x gene


# generate samples
Y = bernoulli(beta).rvs()
obs = pd.DataFrame(X1, columns=[f"dim{j}" for j in range(n_feature1)]) # cell x feature
adata = anndata.AnnData(X=Y, obs=obs)

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [17]:
formula = "~ dim0 + dim1 - 1"
bsim = BernoulliCopulaSimulator()
bsim.fit(adata, formula)

print("marginal AIC:", bsim.marginal_aic)
print("marginal BIC:", bsim.marginal_bic)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


marginal AIC: 43508.49609375
marginal BIC: 43732.53125


In [18]:
beta_hat = expit(X1 @ bsim.params['coef_mean'].values)
log_likelihood = np.sum(bernoulli(beta_hat).logpmf(Y))
k = n_feature * n_gene # nparams

print("marginal AIC:", 2 * k - 2 * log_likelihood)
print("marginal BIC:", k * np.log(n_sample) - 2 * log_likelihood)

marginal AIC: 43508.4944355313
marginal BIC: 43732.530533912985


We can see the scdesigner AIC/BIC aligns with AIC/BIC we calculated by hand (using scipy.stats).